In [5]:
# -*- coding: utf-8 -*-
"""
Created on Thu Dec 19 17:08:19 2024

@author: Rodrigo Meira
"""

from libs.compressor_class import *
from libs.compression import *
from libs.gc_eos_soave import *
from libs.viscosity import *
from libs.plenum_system import *
from libs.simulation import *
from libs.eos_database import *

from casadi import *
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import torch

ModuleNotFoundError: No module named 'eos_database'

In [ ]:
list_names = ["CH4", "C2H6", "C3H8", "iC4H10", "nC4H10", "iC5H12", "nC5H12", 
                  "nC6H14", "nC7H16", "nC8H18", "nC9H20", "nC10H22", "nC11H24", 
                   "nC12H26", "nC14H30", "N2", "H2O", "CO2", "C15+"]

nwe = [0.9834, 0.0061, 0.0015, 0.0003, 0.0003, 0.00055, 0.0004, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0003, 0.0, 0.0008, 0.0]

dict_composition= {list_names[i]: nwe[i] for i in range(len(nwe))}

mixture = Mixture(list_of_species, dict_composition)

volumn_desviation = [0] * 19

vlv = valve(kv=0.38)
Vpp = 2.0 
Lc = 2.0 
A1 = 2.6e-3
gas = gc_eos_class(mixture, 300, 4500, None, 1, 0, Aij, volumn_desviation, 'gas')
comp = CompressorClass()
visc = viscosity(mixture, volumn_desviation)
compressor = compression(gas, comp, visc)
plenum_sys = plenum(gas, compressor, vlv, Vpp, Lc, A1)
nAlphas = 6
N_RotS = np.random.uniform(600, 750, nAlphas+1)
alphas = np.random.uniform(0.35,0.65, nAlphas+1)
nData = 2000
dt = 0.1
timestep = 3
x0 = [14.9919, 339.69, 0.42885]
z0 = [6245.39, 6245.39, 321.672, 0.445562, 319.423, 0.503621, 320.097, 0.396345, 339.69, 0.42885, 0.514917]
u0 = [4500, 300, 600, alphas[0], 5000]

#medido: tp, pp, mp, p2, t2, todos u0.

In [ ]:
sim = Simulation(plenum_sys, compressor, x0, z0, u0, nAlphas, alphas, N_RotS, nData, dt, timestep)
x_values, z_values, time_steps, alpha_values, N_values, x_train, y_train, x_min, x_max = sim.run()

In [ ]:
plt.figure(figsize=(12, 6))

plt.figure(figsize=(12, 6))
plt.plot(time_steps, x_values[:, 0], label=f'Vazao Massica')
plt.xlabel('Tempo')
plt.ylabel(f'Vazao massica')
plt.title(f'Evolução da Vazao massica ao Longo do Tempo')
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(time_steps, x_values[:, 1], label=f'Temperatura')
plt.xlabel('Tempo')
plt.ylabel(f'Temperatura')
plt.title(f'Evolução da Temperatura ao Longo do Tempo')
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(time_steps, x_values[:, 2], label=f'Volume')
plt.xlabel('Tempo')
plt.ylabel(f'Volume')
plt.title(f'Evolução do Volume ao Longo do Tempo')
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(time_steps, z_values[:, 0], label='Pp', color='green')
plt.xlabel('Tempo')
plt.ylabel('Estado z[0]')
plt.title('Evolução da Pressao do Plenum ao Longo do Tempo')
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(time_steps, alpha_values, label='Abertura da válvula', color='red')
plt.xlabel('Tempo')
plt.ylabel('Abertura da Válvula')
plt.title('Variação da Abertura da Válvula ao Longo do Tempo')
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(time_steps, N_values, label='Rotacao', color='red')
plt.xlabel('Tempo')
plt.ylabel('Rotacao')
plt.title('Variação da Rotacao ao Longo do Tempo')
plt.legend()
plt.grid()
plt.show()